# ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [149]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [150]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [151]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [152]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

12


## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [153]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [154]:
# Create the sparkifydb keyspace

session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
""")

#### Set Keyspace

In [155]:
# set sparkifydb keyspace
session.set_keyspace('sparkifydb')

## Data modeling in Apache Cassandra:
   - 1 table per 1 query
   - knowing your queries first and modeling to those queries is essential
   - denormalization of tables in data modeling is required

### Queries:

#### 1. Fetch artist, song title and length for session id = 338 and item in session = 4
I'm creating a table `artist_song_length` in the sparkifydb Keyspace. Assuming that each `itemInSession` is unique for a given `sessionId`, partitioning on `sessionId` and adding `itemNumberInSession` as the clustering column will create a unique row-key/primary key.

In [165]:
# Create table artist_song_length 
query = "CREATE TABLE IF NOT EXISTS artist_song_length "
query = query + "(session_id int, item_in_session int, artist text, song_title text, length float, PRIMARY KEY(session_id, item_in_session))"
session.execute(query)

In [168]:
# Insert rows into artist_song_length
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_length (session_id, item_in_session, artist, song_title, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))

In [169]:
query = "SELECT artist, song_title, length FROM artist_song_length WHERE session_id = 338 AND item_in_session = 4 "
result = session.execute(query)

# Print results
for r in result: 
    print(r.artist, r.song_title, r.length)

#### 2. Fetch artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
I'm creating a table `artist_song_user` in the sparkifydb Keyspace. Further, combining the `userId` and `seesionId` in a composite primary key to make sure the row-key is unique. I'm using `itemInSession` as a clustering column to apply sorting for the specified query.

In [170]:
# Create table artist_song_user 
query = "CREATE TABLE IF NOT EXISTS artist_song_user "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song_title text, first_name text, last_name text,  \
                  PRIMARY KEY((user_id, session_id), item_in_session))"
session.execute(query)

In [171]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_song_user (user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[-1]), int(line[8]), int(line[3]), str(line[0]), str(line[9]), str(line[1]), str(line[4])))

In [172]:
query = "SELECT artist, song_title, first_name, last_name FROM artist_song_user WHERE user_id = 10 AND session_id = 182 "
result = session.execute(query)

# Print results
for r in result: 
    print(r.artist, r.song_title, r.first_name, r.last_name )

#### 3. Fetch every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
Creating a table `user_song_title` in the sparkifydb Keyspace. Having song title as the partition key and user id as the clustering column will make a unique row for every user who've been listening to a given song. 

In [173]:
# Create table user_song_title 
query = "CREATE TABLE IF NOT EXISTS user_song_title "
query = query + "(song_title text, user_id int, first_name text, last_name text, \
                  PRIMARY KEY(song_title, user_id))"
session.execute(query)

In [174]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song_title (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (str(line[-2]), int(line[-1]), str(line[1]), str(line[4])))

In [175]:
query = "SELECT * FROM user_song_title WHERE song_title = 'All Hands Against His Own' "
result = session.execute(query)

# Print results
for r in result: 
    print(r.user_id, r.first_name, r.last_name, r.song_title )

### Drop the tables before closing out the sessions

In [176]:
# Drop table artist_song_length
query = "DROP TABLE artist_song_length"
session.execute(query)

# Drop table artist_song_user
query = "DROP TABLE artist_song_user"
session.execute(query)

# Drop table user_song_title
query = "DROP TABLE user_song_title"
session.execute(query)

### Close the session and cluster connection¶

In [177]:
session.shutdown()
cluster.shutdown()